In [72]:
import formulas

from ast_building.formula_parser import FormulaParser

In [37]:
ast = formulas.Parser().ast('=B2+SUM(A1:A3)')


In [69]:
type(ast[0][0])
type(ast[1])

formulas.builder.AstBuilder

In [70]:
type(ast[0][0])


formulas.tokens.operand.Range

In [47]:
pipe = ast[1].compile()

In [59]:
pipe.inputs

OrderedDict([('A1:A3', <Ranges>(A1:A3)), ('B2', <Ranges>(B2))])

In [57]:
pipe(1,2)

Array(3.0, dtype=object)

Get inputs delta using the above method.

Evaluate by passing in the inputs into the pipe() function


SIMPLIFIES EVERYTHING SIGNIFICANTLY

In [78]:
formula_1 = "=SUM(A1:A3) + 5 - A2"
formula_2 = "=SUM(A1:A3) + 5 - A3"

In [82]:
formula_1_ast = FormulaParser.parse_formula(formula_1)
formula_2_ast = FormulaParser.parse_formula(formula_2)

In [86]:
formula_1_elements_list = formula_1_ast[0]

In [87]:
formula_2_elements_list = formula_2_ast[0]

In [104]:
formula_1_elements_list

[SUM <Function>,
 ( <Parenthesis>,
 A1:A3 <Range>,
 ) <Parenthesis>,
 + <Operator>,
 5 <Number>,
 - <Operator>,
 A2 <Range>]

In [116]:
from formulas.tokens.operand import Range
from excel_utils import ExcelUtils


class FormulaGenerator:

    @staticmethod
    def get_delta_between_coordinates(coordinate1, coordinate2):
        cell_start_column2, cell_start_row2, cell_end_column2, cell_end_row2, _ = ExcelUtils.get_coordinates_from_range(coordinate2)
        cell_start_column1, cell_start_row1, cell_end_column1, cell_end_row1, _ = ExcelUtils.get_coordinates_from_range(coordinate1)

        cell_start_column_delta = cell_start_column2 - cell_start_column1
        cell_start_row_delta = cell_start_row2 - cell_start_row1
        cell_end_column_delta = cell_end_column2 - cell_end_column1
        cell_end_row_delta = cell_end_row2 - cell_end_row1

        return cell_start_column_delta, cell_start_row_delta, cell_end_column_delta, cell_end_row_delta
    
    @staticmethod
    def generate_generic_formula(formula_1_elements_list, formula_2_elements_list):
                
        delta_dictionary = {}

        for index, (element1, element2) in enumerate(zip(formula_1_elements_list, formula_2_elements_list)):
            
            if isinstance(element1, Range) and isinstance(element2, Range):                            

                delta = FormulaGenerator.get_delta_between_coordinates(element1.name, element2.name)
                delta_dictionary[index] = delta

        return delta_dictionary

In [115]:
delta_dictionary

{2: (0, 0, 0, 0), 7: (0, 1, 0, 1)}

In [101]:
type(formula_1_elements_list[2])

formulas.tokens.operand.Range

In [97]:
formula_1_elements_list[2].name

'A1:A3'

In [89]:
formula_2_elements_list

[SUM <Function>,
 ( <Parenthesis>,
 A1:A3 <Range>,
 ) <Parenthesis>,
 + <Operator>,
 5 <Number>,
 - <Operator>,
 A3 <Range>]